In [1]:
import mysql.connector
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from pandas import Timestamp
from IPython.display import display
from Functions.connection.connection import *
from Functions.AlertP1.data_cleaning import *
from Functions.AlertP1.features import *
from Functions.analysis.step_analysis import *
from Functions.AlertP1.dummy_features import *
from Functions.Models.decision_tree import *
from Functions.Models.Logistic_regression import *
from Functions.Models.evaluation import *

creds = ["uktuserdaroglu","Vacy4201(Pri","172.20.20.4","hgo",3306]
#Connection to the database
host = creds[2]
user = creds[0]
password = creds[1]
database = creds[3]
port = creds[4]
mydb = mysql.connector.connect(host=host, user=user, database=database, port=port, password=password, auth_plugin='mysql_native_password')
mycursor = mydb.cursor()

#Safecheck to guarantee that the connection worked
mycursor.execute('SHOW TABLES;')
print(f"Tables: {mycursor.fetchall()}")
print(mydb.connection_id) #it'll give connection_id,if got connected

Tables: [('ConsultaUrgencia_doentespedidosconsultaNeurologia2012',), ('consultaneurologia2012',), ('consultaneurologia201216anon_true',)]
188


In [2]:
alertP = pd.read_sql("""SELECT * FROM consultaneurologia201216anon_true""",mydb)
alertP1=alertP.copy()
alertP1
alertP1.info()


/Users/utkuserdaroglu/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10150 entries, 0 to 10149
Data columns (total 31 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID_DOENTE            10150 non-null  object
 1   PROCESSO             10150 non-null  object
 2   COD_REFERENCIA       10150 non-null  object
 3   COD_PZ               10150 non-null  object
 4   COD_UNID_SAUDE_PROV  10150 non-null  int64 
 5   UNID_PROV            10150 non-null  object
 6   TIPO_UNID            10150 non-null  object
 7   PROVENIENCIA         10150 non-null  object
 8   COD_CTH_PRIOR        10150 non-null  object
 9   CTH_PRIOR            10150 non-null  object
 10  COD_MOTIVO_RECUSA    10150 non-null  int64 
 11  DES_MOTIVO_RECUSA    10150 non-null  object
 12  COD_ESPECIALIDADE    10150 non-null  int64 
 13  DES_ESPECIALIDADE    10150 non-null  object
 14  agrupadora           10150 non-null  object
 15  OUTRA_ENTIDADE       10150 non-null  int64 
 16  DATA

In [3]:
#changing columns with date to a datetime
date_format_alertP1(alertP1)
#change the blank values in proveniencia and priority to unknown
replace_blank(alertP1)
#create a column with the output of a referral and select the refused and accepted ones
alertP1=result(alertP1)

/Users/utkuserdaroglu/Desktop/utkubranch/PBL-HGO/Functions/AlertP1/data_cleaning.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alertP1['PROVENIENCIA'][alertP1['PROVENIENCIA']=='']='unknown'
/Users/utkuserdaroglu/Desktop/utkubranch/PBL-HGO/Functions/AlertP1/data_cleaning.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alertP1['CTH_PRIOR'][alertP1['CTH_PRIOR']=='']='unknown'


In [4]:
class_area(alertP1)
text_length(alertP1)
referral_steps(alertP1)
speciality(alertP1)
alertP1=unit(alertP1)
alertP1=alertP1.sort_values('DATA_RECEPCAO')
alertP1

/Users/utkuserdaroglu/Desktop/utkubranch/PBL-HGO/Functions/AlertP1/features.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alertP1['step'][alertP1['step']>=3]='3+'
/Users/utkuserdaroglu/Desktop/utkubranch/PBL-HGO/Functions/AlertP1/features.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alertP1['unit'][alertP1['TIPO_UNID']!='CS/USF']='HOSP'


,ID_DOENTE,PROCESSO,COD_REFERENCIA,COD_PZ,COD_UNID_SAUDE_PROV,UNID_PROV,TIPO_UNID,PROVENIENCIA,COD_CTH_PRIOR,CTH_PRIOR,...,resume saída,mês_saida,ano_saida,Texto,result,area,text_length,step,speciality_type,unit
9915,SEAFHB,BBLVVLDC,LOHHHLOV,SFGHADG,3150305,UCSP RAINHA D. LEONOR,CS/USF,CTH,,unknown,...,30/03/2012,3,2012,,1,inside area,0,1,Other specialities,UCSP
6944,ZDGGGA,BMJNMVVJ,LOHHHHRS,,0,,OUTRA,unknown,,unknown,...,01/08/2012,8,2012,,1,outside area,0,1,General Neurology,HOSP
6542,ZZZBGG,BMJIBILB,LOHHHOHO,SSBSEGA,3150301,FRANCISCO X NORONHA,CS/USF,CTH,,unknown,...,10/01/2012,1,2012,,0,outside area,0,1,General Neurology,CS
3067,DSHCGG,BMIBJIE,LOHHHOUQ,SFAAECF,3151672,USF AMORA SAUDAVEL,CS/USF,CTH,,unknown,...,12/03/2012,3,2012,,1,inside area,0,1,Other specialities,USF
2078,AESBSC,NINBJJC,LOHHHPTO,,0,,OUTRA,unknown,,unknown,...,22/01/2013,1,2013,,1,outside area,0,1,General Neurology,HOSP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7353,ZSGCCF,BMJCJLIJ,LSHVLHUO,ZEFHCECF,3150672,USF FEIJO-CS COVA DA PIEDADE,CS/USF,CTH,3,Normal,...,20/03/2017,3,2017,,1,inside area,0,2,General Neurology,CS
4044,CDABCC,VLNVILNB,LSHVLLUO,ZEFZDDBB,3150371,USF S JOAO DO PRAGAL-CS ALMADA,CS/USF,CTH,3,Normal,...,21/03/2017,3,2017,,1,inside area,0,1,General Neurology,USF
6424,FCDEF,BIJIMMIJ,LSHVLOQU,,0,,OUTRA,SAM,3,Normal,...,01/03/2018,3,2018,,1,outside area,0,2,Other specialities,HOSP
9425,EBESGE,BDVDJJIB,LSHVLLVV,,0,,OUTRA,SAM,,unknown,...,04/09/2017,9,2017,,1,outside area,0,1,Other specialities,HOSP


In [5]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
alertP1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9374 entries, 9915 to 5513
Data columns (total 37 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID_DOENTE            9374 non-null   object        
 1   PROCESSO             9374 non-null   object        
 2   COD_REFERENCIA       9374 non-null   object        
 3   COD_PZ               9374 non-null   object        
 4   COD_UNID_SAUDE_PROV  9374 non-null   int64         
 5   UNID_PROV            9374 non-null   object        
 6   TIPO_UNID            9374 non-null   object        
 7   PROVENIENCIA         9374 non-null   object        
 8   COD_CTH_PRIOR        9374 non-null   object        
 9   CTH_PRIOR            9374 non-null   object        
 10  COD_MOTIVO_RECUSA    9374 non-null   int64         
 11  DES_MOTIVO_RECUSA    9374 non-null   object        
 12  COD_ESPECIALIDADE    9374 non-null   int64         
 13  DES_ESPECIALIDADE    9374 non-

In [6]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
alertP1.head()

,ID_DOENTE,PROCESSO,COD_REFERENCIA,COD_PZ,COD_UNID_SAUDE_PROV,UNID_PROV,TIPO_UNID,PROVENIENCIA,COD_CTH_PRIOR,CTH_PRIOR,...,resume saída,mês_saida,ano_saida,Texto,result,area,text_length,step,speciality_type,unit
9915,SEAFHB,BBLVVLDC,LOHHHLOV,SFGHADG,3150305,UCSP RAINHA D. LEONOR,CS/USF,CTH,,unknown,...,30/03/2012,3,2012,,1,inside area,0,1,Other specialities,UCSP
6944,ZDGGGA,BMJNMVVJ,LOHHHHRS,,0,,OUTRA,unknown,,unknown,...,01/08/2012,8,2012,,1,outside area,0,1,General Neurology,HOSP
6542,ZZZBGG,BMJIBILB,LOHHHOHO,SSBSEGA,3150301,FRANCISCO X NORONHA,CS/USF,CTH,,unknown,...,10/01/2012,1,2012,,0,outside area,0,1,General Neurology,CS
3067,DSHCGG,BMIBJIE,LOHHHOUQ,SFAAECF,3151672,USF AMORA SAUDAVEL,CS/USF,CTH,,unknown,...,12/03/2012,3,2012,,1,inside area,0,1,Other specialities,USF
2078,AESBSC,NINBJJC,LOHHHPTO,,0,,OUTRA,unknown,,unknown,...,22/01/2013,1,2013,,1,outside area,0,1,General Neurology,HOSP


In [7]:
all=alertP1[["ID_DOENTE","DATA_RECEPCAO","PROVENIENCIA","COD_MOTIVO_RECUSA","DES_MOTIVO_RECUSA","DES_ESPECIALIDADE","result","area","text_length","step","unit"]]
all["total"]=1
all["result"]=all["result"].astype(int)
all.info()

/var/folders/h9/v0fvmkcx2fg901hrz6xs3x9c0000gn/T/ipykernel_9030/2184699601.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all["total"]=1
/var/folders/h9/v0fvmkcx2fg901hrz6xs3x9c0000gn/T/ipykernel_9030/2184699601.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all["result"]=all["result"].astype(int)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 9374 entries, 9915 to 5513
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   ID_DOENTE          9374 non-null   object        
 1   DATA_RECEPCAO      9374 non-null   datetime64[ns]
 2   PROVENIENCIA       9374 non-null   object        
 3   COD_MOTIVO_RECUSA  9374 non-null   int64         
 4   DES_MOTIVO_RECUSA  9374 non-null   object        
 5   DES_ESPECIALIDADE  9374 non-null   object        
 6   result             9374 non-null   int64         
 7   area               9374 non-null   object        
 8   text_length        9374 non-null   int64         
 9   step               9374 non-null   object        
 10  unit               9374 non-null   object        
 11  total              9374 non-null   int64         
dtypes: datetime64[ns](1), int64(4), object(7)
memory usage: 952.0+ KB


In [32]:
#all dataset
unit=all.groupby("unit")["result","total"].sum()
unit["acc_rate"]=unit["result"]/unit["total"]

unit

/var/folders/h9/v0fvmkcx2fg901hrz6xs3x9c0000gn/T/ipykernel_9030/3846134350.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  unit=all.groupby("unit")["result","total"].sum()


,result,total,acc_rate
unit,,,
CS,1330,1796,0.740535
HOSP,1887,3801,0.496448
UCSP,499,704,0.708807
USF,2538,3073,0.825903


In [30]:
#text lenght >1, 
unit_t=all[all["text_length"]>0]
unit_t=unit_t.groupby("unit")["result","total"].sum()
unit_t["acc_rate"]=unit_t["result"]/unit_t["total"]
unit_t


/var/folders/h9/v0fvmkcx2fg901hrz6xs3x9c0000gn/T/ipykernel_9030/1855657030.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  unit_t=unit_t.groupby("unit")["result","total"].sum()


,result,total,acc_rate
unit,,,
CS,233,302,0.771523
HOSP,340,657,0.517504
UCSP,87,119,0.731092
USF,439,531,0.826742


In [33]:
#all dataset
spec_a=all.groupby("DES_ESPECIALIDADE")["result","total"].sum()
spec_a["acc_rate"]=spec_a["result"]/spec_a["total"]
spec_a

/var/folders/h9/v0fvmkcx2fg901hrz6xs3x9c0000gn/T/ipykernel_9030/3168660096.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  spec_a=all.groupby("DES_ESPECIALIDADE")["result","total"].sum()


,result,total,acc_rate
DES_ESPECIALIDADE,,,
APOIO DOENCAS NEUROLOGICAS CRONICAS,14,14,1.000000
DOENCAS DO MOVIMENTO,253,254,0.996063
DVC,63,66,0.954545
EPILEPSIA,370,378,0.978836
NEUROLOGIA,5168,8264,0.625363
NEUROLOGIA - CEFALEIAS,2,2,1.000000
NEUROLOGIA - ESCLEROSE MULTIPLA,52,53,0.981132
NEUROLOGIA DO COMPORTAMENTO,332,343,0.967930


In [19]:
#text lenght >1, 
spec=all[all["text_length"]>0]
spec=spec.groupby("DES_ESPECIALIDADE")["result","total"].sum()
spec["acc_rate"]=spec["result"]/spec["total"]
spec





/var/folders/h9/v0fvmkcx2fg901hrz6xs3x9c0000gn/T/ipykernel_9030/2530032183.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  spec=spec.groupby("DES_ESPECIALIDADE")["result","total"].sum()


,result,total,acc_rate
DES_ESPECIALIDADE,,,
APOIO DOENCAS NEUROLOGICAS CRONICAS,1,1,1.000000
DOENCAS DO MOVIMENTO,50,51,0.980392
DVC,9,9,1.000000
EPILEPSIA,69,69,1.000000
NEUROLOGIA,904,1411,0.640680
NEUROLOGIA - CEFALEIAS,1,1,1.000000
NEUROLOGIA - ESCLEROSE MULTIPLA,7,7,1.000000
NEUROLOGIA DO COMPORTAMENTO,58,60,0.966667


In [36]:
#all dataset
step_a=all.groupby("step")["result","total"].sum()
step_a["acc_rate"]=step_a["result"]/step_a["total"]
step_a.reset_index()

/var/folders/h9/v0fvmkcx2fg901hrz6xs3x9c0000gn/T/ipykernel_9030/2832888825.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  step_a=all.groupby("step")["result","total"].sum()


,step,result,total,acc_rate
0,1,5164,7885,0.654914
1,2,938,1276,0.735110
2,3+,152,213,0.713615


In [37]:
#text lenght >1, 
step_t=all[all["text_length"]>0]
step_t=step_t.groupby("step")["result","total"].sum()
step_t["acc_rate"]=step_t["result"]/step_t["total"]
step_t.reset_index()

/var/folders/h9/v0fvmkcx2fg901hrz6xs3x9c0000gn/T/ipykernel_9030/3273770789.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  step_t=step_t.groupby("step")["result","total"].sum()


,step,result,total,acc_rate
0,1,882,1318,0.669196
1,2,180,244,0.737705
2,3+,37,47,0.787234


In [38]:
#all dataset
area_a=all.groupby("area")["result","total"].sum()
area_a["acc_rate"]=area_a["result"]/area_a["total"]
area_a.reset_index()

/var/folders/h9/v0fvmkcx2fg901hrz6xs3x9c0000gn/T/ipykernel_9030/3005475344.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  area_a=all.groupby("area")["result","total"].sum()


,area,result,total,acc_rate
0,inside area,4111,5197,0.791033
1,outside area,2143,4177,0.513048


In [40]:
#text lenght >1, 
area_t=all[all["text_length"]>0]
area_t=area_t.groupby("area")["result","total"].sum()
area_t["acc_rate"]=area_t["result"]/area_t["total"]
area_t.reset_index()

/var/folders/h9/v0fvmkcx2fg901hrz6xs3x9c0000gn/T/ipykernel_9030/421145237.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  area_t=area_t.groupby("area")["result","total"].sum()


,area,result,total,acc_rate
0,inside area,714,900,0.793333
1,outside area,385,709,0.543018


In [43]:
#all dataset
res_a=all.groupby("result")["total"].sum()

res_a.reset_index()

,result,total
0,0,3120
1,1,6254


In [45]:
#text lenght >1, 
res_t=all[all["text_length"]>0]
res_t=res_t.groupby("result")["total"].sum()
res_t.reset_index()

,result,total
0,0,510
1,1,1099


In [46]:
#all dataset
prov_a=all.groupby("PROVENIENCIA")["result","total"].sum()
prov_a["acc_rate"]=prov_a["result"]/prov_a["total"]
prov_a.reset_index()

/var/folders/h9/v0fvmkcx2fg901hrz6xs3x9c0000gn/T/ipykernel_9030/1001866789.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  prov_a=all.groupby("PROVENIENCIA")["result","total"].sum()


,PROVENIENCIA,result,total,acc_rate
0,CTH,4368,5565,0.784906
1,SAM,1493,2467,0.605188
2,SON,41,242,0.169421
3,unknown,352,1100,0.320000


In [47]:
#text lenght >1, 
prov_t=all[all["text_length"]>0]
prov_t=prov_t.groupby("PROVENIENCIA")["result","total"].sum()
prov_t["acc_rate"]=prov_t["result"]/prov_t["total"]
prov_t.reset_index()

/var/folders/h9/v0fvmkcx2fg901hrz6xs3x9c0000gn/T/ipykernel_9030/4266282806.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  prov_t=prov_t.groupby("PROVENIENCIA")["result","total"].sum()


,PROVENIENCIA,result,total,acc_rate
0,CTH,760,953,0.797482
1,SAM,267,438,0.609589
2,SON,12,30,0.400000
3,unknown,60,188,0.319149


In [12]:
epilepsia=all[all["DES_ESPECIALIDADE"]=="EPILEPSIA"]
epilepsia=epilepsia[epilepsia["text_length"]>0]
epilepsia["result"].value_counts()
epilepsia

1    69
Name: result, dtype: int64

,ID_DOENTE,DATA_RECEPCAO,PROVENIENCIA,COD_MOTIVO_RECUSA,DES_MOTIVO_RECUSA,DES_ESPECIALIDADE,result,area,text_length,step,unit,total
985,BSEZF,2012-01-11,CTH,53,Falta Injustificada do Utente,EPILEPSIA,1,inside area,401,1,USF,1
884,FDAAFS,2012-01-24,CTH,0,,EPILEPSIA,1,inside area,424,1,USF,1
689,DABEC,2012-02-08,CTH,0,,EPILEPSIA,1,inside area,161,1,UCSP,1
1154,FSBBFF,2012-03-01,unknown,0,,EPILEPSIA,1,outside area,10,1,HOSP,1
1426,SESHFG,2012-03-21,CTH,53,Falta Injustificada do Utente,EPILEPSIA,1,inside area,246,1,USF,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1195,FAHSEH,2015-12-14,SAM,0,,EPILEPSIA,1,outside area,167,1,HOSP,1
1448,FBFFSS,2016-03-15,CTH,0,,EPILEPSIA,1,inside area,539,1,USF,1
1018,FSZBAC,2016-05-10,SAM,0,,EPILEPSIA,1,outside area,27,1,HOSP,1
975,SAEHEB,2016-06-08,SAM,0,,EPILEPSIA,1,outside area,31,3+,HOSP,1


In [13]:

b=all[all["text_length"]>0]
c=b.groupby(["DES_ESPECIALIDADE","COD_MOTIVO_RECUSA","DES_MOTIVO_RECUSA"])["result","total"].sum()
c.reset_index()
6254/9260

/var/folders/h9/v0fvmkcx2fg901hrz6xs3x9c0000gn/T/ipykernel_9030/3405078654.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  c=b.groupby(["DES_ESPECIALIDADE","COD_MOTIVO_RECUSA","DES_MOTIVO_RECUSA"])["result","total"].sum()


,DES_ESPECIALIDADE,COD_MOTIVO_RECUSA,DES_MOTIVO_RECUSA,result,total
0,APOIO DOENCAS NEUROLOGICAS CRONICAS,0,,1,1
1,DOENCAS DO MOVIMENTO,0,,38,38
2,DOENCAS DO MOVIMENTO,2,Utente fora da area,0,1
3,DOENCAS DO MOVIMENTO,15,Consulta Marcada. O Doente Desistiu,1,1
4,DOENCAS DO MOVIMENTO,53,Falta Injustificada do Utente,11,11
5,DVC,0,,8,8
6,DVC,15,Consulta Marcada. O Doente Desistiu,1,1
7,EPILEPSIA,0,,57,57
8,EPILEPSIA,12,Marcada em Subsequentes,1,1
9,EPILEPSIA,13,Marcada sem Referencia,1,1


0.675377969762419

In [14]:
#double check if th
#with text how many have data recepcao and that kind of datas. Compare it with full sample and ones in. 


<class 'pandas.core.frame.DataFrame'>
Int64Index: 9374 entries, 9915 to 5513
Data columns (total 37 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID_DOENTE            9374 non-null   object        
 1   PROCESSO             9374 non-null   object        
 2   COD_REFERENCIA       9374 non-null   object        
 3   COD_PZ               9374 non-null   object        
 4   COD_UNID_SAUDE_PROV  9374 non-null   int64         
 5   UNID_PROV            9374 non-null   object        
 6   TIPO_UNID            9374 non-null   object        
 7   PROVENIENCIA         9374 non-null   object        
 8   COD_CTH_PRIOR        9374 non-null   object        
 9   CTH_PRIOR            9374 non-null   object        
 10  COD_MOTIVO_RECUSA    9374 non-null   int64         
 11  DES_MOTIVO_RECUSA    9374 non-null   object        
 12  COD_ESPECIALIDADE    9374 non-null   int64         
 13  DES_ESPECIALIDADE    9374 non-